In [8]:

import math
import numpy as np
import pandas as pd

# import scikitplot
import seaborn as sns
from matplotlib import pyplot

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, Activation
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.utils import np_utils

In [9]:
emotion_label_to_text = {
  0: 'Angry',
  1: 'Fear',
  2: 'Happy',
  3: 'Neutral',
  4: 'Sadness',

}

In [7]:
!pip install scikitplot

ERROR: Could not find a version that satisfies the requirement scikitplot (from versions: none)
ERROR: No matching distribution found for scikitplot


In [3]:
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
import numpy as np

In [30]:


train_dir = "D:\\Facial_emotional_recognition\\fer\\training"
valid_dir = "D:\\Facial_emotional_recognition\\fer\\validation"
test_dir = "D:\\Facial_emotional_recognition\\fer\\test"

train_datagen = ImageDataGenerator(rescale = 1./255,
                                      rotation_range=15,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      shear_range=0.2,
                                      zoom_range=0.2,
                                      horizontal_flip=True)
#                                       fill_mode='nearest')


valid_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(48,48),
                                                    color_mode="grayscale",
                                                    class_mode='categorical',
                                                    batch_size=batch_size)

valid_generator = valid_datagen.flow_from_directory(valid_dir,
                                                    target_size=(48,48),
                                                    color_mode="grayscale",
                                                    class_mode='categorical',
                                                    batch_size=batch_size)

test_generator = test_datagen.flow_from_directory( test_dir,
                                                    target_size=(48, 48),
                                                    color_mode="grayscale",
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True,
                                                    seed=42)

Found 6312 images belonging to 5 classes.
Found 511 images belonging to 5 classes.
Found 251 images belonging to 5 classes.


In [31]:
num_classes = 5
batch_size = 64

img_width = 48
img_height =48
img_depth = 1


In [28]:
def build_net(optim, num_classes):
    
    net = Sequential(name='DCNN')
    
    net.add(Conv2D(filters=64,kernel_size=(5,5),input_shape=(img_width, img_height, img_depth),activation='elu',padding='same',kernel_initializer='he_normal',name='conv2d_1'))
    net.add(BatchNormalization(name='batchnorm_1'))
    
    net.add(Conv2D(filters=64, kernel_size=(5,5),activation='elu',padding='same',kernel_initializer='he_normal',name='conv2d_2'))
    net.add(BatchNormalization(name='batchnorm_2'))    
    net.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_1'))
    net.add(Dropout(0.4, name='dropout_1'))

    net.add(Conv2D(filters=128,kernel_size=(3,3),activation='elu',padding='same',kernel_initializer='he_normal',name='conv2d_3'))
    net.add(BatchNormalization(name='batchnorm_3'))
    net.add( Conv2D(filters=128, kernel_size=(3,3),activation='elu', padding='same',kernel_initializer='he_normal',name='conv2d_4'))
    net.add(BatchNormalization(name='batchnorm_4'))   
    net.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_2'))
    net.add(Dropout(0.4, name='dropout_2'))

    net.add(Conv2D(filters=256, kernel_size=(3,3),activation='elu',padding='same',kernel_initializer='he_normal',name='conv2d_5'))
    net.add(BatchNormalization(name='batchnorm_5'))
    net.add(Conv2D(filters=256, kernel_size=(3,3), activation='elu',padding='same',kernel_initializer='he_normal', name='conv2d_6'))
    net.add(BatchNormalization(name='batchnorm_6'))
    
    net.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_3'))
    net.add(Dropout(0.5, name='dropout_3'))

    net.add(Flatten(name='flatten'))
        
    net.add(Dense(128, activation='elu', kernel_initializer='he_normal', name='dense_1'))
    net.add(BatchNormalization(name='batchnorm_7'))
    
    net.add(Dropout(0.6, name='dropout_4'))
    
    net.add(Dense(num_classes,activation='softmax',name='out_layer'))
    
    net.compile(
        loss='categorical_crossentropy',
        optimizer=optim,
        metrics=['accuracy']
    )
    
    net.summary()
    
    return net

In [19]:
early_stopping = EarlyStopping(monitor='val_accuracy',
                                min_delta=0.00005,
                                patience=11,
                                verbose=1,
                                restore_best_weights=True)

lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy',
                                factor=0.5,
                                patience=7,
                                min_lr=1e-7,
                                verbose=1)

callbacks = [early_stopping,lr_scheduler,]

In [22]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

In [23]:
batch_size = 32 #batch size of 32 performs the best.

epochs = 100

optims = [
  optimizers.Nadam(
      learning_rate=0.001,
      beta_1=0.9,
      beta_2=0.999,
      epsilon=1e-07,
      name='Nadam'),
    
  optimizers.Adam(0.001),
]

In [ ]:
model = build_net(optims[1], num_classes) 

history = model.fit_generator(
    train_generator,
    validation_data = valid_generator,
    steps_per_epoch=STEP_SIZE_TRAIN,
    epochs=epochs,
    callbacks=callbacks,
    verbose=1,
    use_multiprocessing=True
)



Model: "DCNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 64)        1664      
_________________________________________________________________
batchnorm_1 (BatchNormalizat (None, 48, 48, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 64)        102464    
_________________________________________________________________
batchnorm_2 (BatchNormalizat (None, 48, 48, 64)        256       
_________________________________________________________________
maxpool2d_1 (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 128)       73856  

Exception in thread Thread-142:
Traceback (most recent call last):
  File "C:\Users\SHIVAM\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\SHIVAM\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\SHIVAM\anaconda3\lib\site-packages\tensorflow\python\keras\utils\data_utils.py", line 726, in _run
    with closing(self.executor_fn(_SHARED_SEQUENCES)) as executor:
  File "C:\Users\SHIVAM\anaconda3\lib\site-packages\tensorflow\python\keras\utils\data_utils.py", line 703, in pool_fn
    pool = get_pool_class(True)(
  File "C:\Users\SHIVAM\anaconda3\lib\multiprocessing\context.py", line 119, in Pool
    return Pool(processes, initializer, initargs, maxtasksperchild,
  File "C:\Users\SHIVAM\anaconda3\lib\multiprocessing\pool.py", line 212, in __init__
    self._repopulate_pool()
  File "C:\Users\SHIVAM\anaconda3\lib\multiprocessing\pool.py", line 303, in _repopulate_pool
    return self._repopu